# ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ
## ФОРМАТ DATETIME
### В Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.
### В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [2]:
import pandas as pd

melb_data = pd.read_csv('melb_data_ps.csv')
melb_df = melb_data.copy()


In [3]:
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

### Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [5]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

### В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

### Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64. Рассмотрим несколько возможностей этого типа данных.

# ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

## Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

### date — дата;
### year, month, day — год, месяц, день;
### time — время;
### hour, minute, second — час, минута, секунда;
### dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
### day_name — название дня недели;
### dayofyear — порядковый день года;
### quarter — квартал (интервал в три месяца).

## Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью pd.to_datetime().

## Oбратите внимание, что вы не сможете обратиться к аксессору, если ваш столбец не приведён к типу datetime.

### Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [9]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


### В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.
### Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.


### Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [10]:
melb_df['Monthsale'] = melb_df['Date'].dt.month
melb_df['Monthsale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: Monthsale, dtype: float64

# РАБОТА С ИНТЕРВАЛАМИ
### Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [11]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

### В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

### Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [12]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

### Cоздадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [13]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

### Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

### На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [17]:
melb_df = melb_df.drop('YearBuilt', axis=1)

In [27]:
melb_df['WeekDaySale'] = melb_df['Date'].dt.day_of_week
weekend_count = ((melb_df['WeekDaySale'] == 5) + (melb_df['WeekDaySale'] == 6)).value_counts()
print(weekend_count)

True     12822
False      758
Name: WeekDaySale, dtype: int64


In [28]:
ufo_df = pd.read_csv('ufo.csv')
display(ufo_df)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


In [29]:
ufo_df['Time'] = pd.to_datetime(ufo_df['Time'])
display(ufo_df)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,1930-06-01 22:00:00
1,Willingboro,NaN,OTHER,NJ,1930-06-30 20:00:00
2,Holyoke,NaN,OVAL,CO,1931-02-15 14:00:00
3,Abilene,NaN,DISK,KS,1931-06-01 13:00:00
4,New York Worlds Fair,NaN,LIGHT,NY,1933-04-18 19:00:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,2000-12-31 23:00:00
18237,Spirit Lake,NaN,DISK,IA,2000-12-31 23:00:00
18238,Eagle River,NaN,NaN,WI,2000-12-31 23:45:00
18239,Eagle River,RED,LIGHT,WI,2000-12-31 23:45:00


In [31]:
years_max = ufo_df['Time'].dt.year
print(years_max.value_counts(normalize=True))

1999    0.152075
2000    0.144455
1998    0.095554
1995    0.073680
1997    0.067814
          ...   
1936    0.000110
1930    0.000110
1935    0.000055
1934    0.000055
1933    0.000055
Name: Time, Length: 68, dtype: float64


In [33]:
ufo_df['Date'] = ufo_df['Time'].dt.date


In [45]:
NV_col = ufo_df[ufo_df['State'] == 'NV']['Date'].diff()
display(NV_col.mean())
# Timedelta('68 days 22:18:13.992932862')

Timedelta('68 days 22:18:13.992932862')